In [3]:
import sys
import pyzed.sl as sl
import cv2

import math
import numpy as np


In [4]:
def Dis_pt2plane(pts, a, b, c, d):
    """
    Compute the distance from points to the plane
    """
    normal = math.sqrt(a*a+b*b+c*c)
    if normal == 0:
        normal = 1
    
    v = np.array([a,b,c])
    dis = abs(np.dot(pts,v.T)+d)/normal
    return dis

def get_Plane(sampts):
    """
    Compute the equation of the plane
    """
    p1 = sampts[0]
    p2 = sampts[1]
    p3 = sampts[2]
    
    a = ( (p2[1]-p1[1])*(p3[2]-p1[2])-(p2[2]-p1[2])*(p3[1]-p1[1]) )
    b = ( (p2[2]-p1[2])*(p3[0]-p1[0])-(p2[0]-p1[0])*(p3[2]-p1[2]) )
    c = ( (p2[0]-p1[0])*(p3[1]-p1[1])-(p2[1]-p1[1])*(p3[0]-p1[0]) )
    d = ( 0-(a*p1[0]+b*p1[1]+c*p1[2]) )
    
    return a,b,c,d

def Random3points(points3D, ConfidenceIndex):
    """
    Random choose 3 Confidence points
    """
    sample_number = 3
    sample_point_index = random.sample(range(ConfidenceIndex.shape[0]), sample_number)
    sample_points = np.zeros((sample_number,3))
    for i in range(sample_number):
        Confidence_point_index = sample_point_index[i]
        index = ConfidenceIndex[Confidence_point_index]
        y = index // points3D.shape[1]
        x = index % points3D.shape[1]
        sample_points[i] = points3D[y][x]
    return sample_points

# def Random3points(points3D):
#     sample_number = 3
#     sample_point_index = random.sample(range(points3D.shape[0]*points3D.shape[1]), sample_number)
#     sample_points = np.zeros((sample_number,3))
#     for i in range(sample_number):
#         index = sample_point_index[i]
#         y = index // points3D.shape[1]
#         x = index % points3D.shape[1]
#         sample_points[i] = points3D[y][x]
#     return sample_points

def get_inliner_num(points3D,a,b,c,d,inliner_threshold):
    """
    Compute the liner points which distance to plane < threshold
    Also get distance from points to the plane (new Depth Image which re-project depth pixels in surface plane)
    """
    inliner_num = 0
    
    dist = Dis_pt2plane(points3D,a,b,c,d)
    inliner_mask = dist < inliner_threshold
    inliner_num = np.sum(inliner_mask)
    return inliner_num, inliner_mask, dist

def RANSAM(points3D, ConfidenceIndex, ransac_iteration = 1000, inliner_threshold = 0.01):
    best_inlinernum = -1
    best_inlinernum = 0
    best_plane = np.zeros((1,4))
    best_depthImage = np.zeros((points3D.shape[0],points3D.shape[1]))
    best_plane_mask = np.zeros((points3D.shape[0],points3D.shape[1]))
#     best_sampts = np.zeros((3,3))
    
#     print(points3D.shape,points3D[80:90,110])
    for i in range(ransac_iteration):
        sampts = Random3points(points3D, ConfidenceIndex)
        a,b,c,d = get_Plane(sampts)
        
        inliner_num, inliner_mask, depthImage = get_inliner_num(points3D,a,b,c,d,inliner_threshold)
        if(inliner_num > best_inlinernum):
            best_inlinernum = inliner_num
            best_plane = np.array([a,b,c,d])
            best_plane_mask = inliner_mask
            best_depthImage = depthImage
#             best_sampts = sampts
            
    print("Inliner Number\n", best_inlinernum)
    print("Inliner plane\n", best_plane)
    return best_plane, best_depthImage, best_plane_mask

In [30]:
def main():

    init = sl.InitParameters()
    init.camera_resolution = sl.RESOLUTION.RESOLUTION_HD720
    init.depth_mode = sl.DEPTH_MODE.DEPTH_MODE_PERFORMANCE

    #read from streaming
#     sys.argv[1] = '127.0.0.1'
#     if (len(sys.argv) > 1) :
#         ip = sys.argv[1]
#         init.set_from_stream(ip)
#     else :
#         print('Usage : python3 streaming_receiver.py ip')
#         exit(1)
        
    #read from SVO file
    filepath = 'record_1017_1.svo'
    print("Reading SVO file: {0}".format(filepath))

    init = sl.InitParameters(svo_input_filename=filepath,svo_real_time_mode=False)

    cam = sl.Camera()
    status = cam.open(init)
    if status != sl.ERROR_CODE.SUCCESS:
        print(repr(status))
        exit(1)

    runtime = sl.RuntimeParameters()
    
    image = sl.Mat() #image
    depth = sl.Mat() #depth map
    depth_for_display = sl.Mat() #depth map,scale its values to [0, 255]
    point_cloud = sl.Mat() #colored point cloud.
    confidence_map = sl.Mat() # confidence_map  [0 ,100], 0 :the best confidence value; 100 :the most unconfident one.

    key = ''
    First = True
    print("  Quit : CTRL+C\n")
    while key != 113:
        err = cam.grab(runtime)
        if (err == sl.ERROR_CODE.SUCCESS) :
            # Retrieve left image
            cam.retrieve_image(image, sl.VIEW.VIEW_LEFT)
            cam.retrieve_image(depth_for_display, sl.VIEW.VIEW_DEPTH)
            
            # Retrieve depth map. Depth is aligned on the left image
            cam.retrieve_measure(depth, sl.MEASURE.MEASURE_DEPTH)
            # Load depth data into a numpy array
            depth_data = depth.get_data()
            
            # Retrieve colored point cloud. Point cloud is aligned on the left image.
            cam.retrieve_measure(point_cloud, sl.MEASURE.MEASURE_XYZRGBA)
            # Load point cloud into a numpy array
            points3D = point_cloud.get_data()
            
            # Retrieve confidence_map.
            cam.retrieve_measure(confidence_map, sl.MEASURE.MEASURE_CONFIDENCE)
            # Load confidence_map into a numpy array
            confidence_img = confidence_map.get_data()
            confidence_index = np.argwhere(confidence_img <= 60)

            
            #####
            if First == True:
                #RANSAM to get surface plane
#                 surface_plane, depthImg, plane_mask = RANSAM(points3D, ConfidenceIndex, ransac_iteration = 500, inliner_threshold = 0.01)
                First = False
                print("firstly RANSAM")
#             else:
#                 #
#                 depthImg = get_depth_map(points3D, surface_plane)
#                 #Canny edge map(infrared image) + threshold based edge map(depth image)
#                 Cannyedges, Threshold_based_edge, Edge_map, grayImage = get_edge_map(grayImage_int32, depthImg)
#                 #Get hight region by Hight and record its position
#                 High_region_Image, high_region_list = get_high_region(depthImg)
#                 #Get Hand mask by Flood fill from high region position with Edge map
#                 Hand_mask_Image = get_Hand_mask(Edge_map, high_region_list, High_region_Image)
#                 #
#                 cnt, contours_image, hand_center, fingertips = find_fingertip(Hand_mask_Image)
#                 # tracking user hand and fingertips
#                 hand_tracking(hand_center, fingertips)
            
            #####
            
            cv2.imshow("ZED", image.get_data())
            cv2.imshow("Depth img", depth_for_display.get_data())
            key = cv2.waitKey(1)
        else :
            key = cv2.waitKey(1)

    cam.close()


In [31]:
if __name__ == "__main__":
    main()

Reading SVO file: record_1017_1.svo
  Quit : CTRL+C

(720, 1280)
firstly RANSAM
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)
(720, 1280)


KeyboardInterrupt: 